### Debug update_namelists

In [5]:
import numpy as np

In [6]:
a = 1

print(f'{a:08}')

00000001


In [4]:
# Read namelist_cfg and extract the variables we need
year_times  = []
start_times = []
end_times   = []
for year in range(1979,2016):
    if year==1979:
        ts=0;te=0;
    ts, te = update_namelist(ts, te, 1979, 2015, 0)
    year_times.append(year)
    start_times.append(ts)
    end_times.append(te)

for year in range(1979,2025):
    ts, te = update_namelist(ts, te, 1979, 2024, 1)
    year_times.append(year)
    start_times.append(ts)
    end_times.append(te)

echo First year of simulation is 1979
1979 1 23360
echo Next year of simulation is 1980
1980 23361 46784
echo Next year of simulation is 1981
1981 46785 70144
echo Next year of simulation is 1982
1982 70145 93504
echo Next year of simulation is 1983
1983 93505 116864
echo Next year of simulation is 1984
1984 116865 140288
echo Next year of simulation is 1985
1985 140289 163648
echo Next year of simulation is 1986
1986 163649 187008
echo Next year of simulation is 1987
1987 187009 210368
echo Next year of simulation is 1988
1988 210369 233792
echo Next year of simulation is 1989
1989 233793 257152
echo Next year of simulation is 1990
1990 257153 280512
echo Next year of simulation is 1991
1991 280513 303872
echo Next year of simulation is 1992
1992 303873 327296
echo Next year of simulation is 1993
1993 327297 350656
echo Next year of simulation is 1994
1994 350657 374016
echo Next year of simulation is 1995
1995 374017 397376
echo Next year of simulation is 1996
1996 397377 420800
echo

In [42]:
for i in range(len(end_times)):
    print(year_times[i], start_times[i], end_times[i], int(end_times[i])-int(start_times[i]))

1979 841537 864896 23359
1980 864897 888320 23423
1981 888321 911680 23359
1982 911681 935040 23359
1983 935041 958400 23359
1984 958401 981824 23423
1985 981825 1005184 23359
1986 1005185 1028544 23359
1987 1028545 1051904 23359
1988 1051905 1075328 23423
1989 1075329 1098688 23359
1990 1098689 1122048 23359
1991 1122049 1145408 23359
1992 1145409 1168832 23423
1993 1168833 1192192 23359
1994 1192193 1215552 23359
1995 1215553 1238912 23359
1996 1238913 1262336 23423
1997 1262337 1285696 23359
1998 1285697 1309056 23359
1999 1309057 1332416 23359
2000 1332417 1355840 23423
2001 1355841 1379200 23359
2002 1379201 1402560 23359
2003 1402561 1425920 23359
2004 1425921 1449344 23423
2005 1449345 1472704 23359
2006 1472705 1496064 23359
2007 1496065 1519424 23359
2008 1519425 1542848 23423
2009 1542849 1566208 23359
2010 1566209 1589568 23359
2011 1589569 1612928 23359
2012 1612929 1636352 23423
2013 1636353 1659712 23359
2014 1659713 1683072 23359
2015 1683073 1706432 23359
2016 1706433 1

In [ ]:
with open('/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/scripts/end_timestep.txt','r') as file:
    prev_round_timestep_end = int(file.readlines()[0])
    print(prev_round_timestep_end)

In [2]:
def next_timestep(year, start_year, tstart, tend, calendar='standard', dt=1350):

    timesteps=0
            
    if year == start_year:
        # New simulation
        tstart = 1
        tend = timesteps_in_year(year, calendar=calendar, dt=dt)
    else:
        for year_tmp in range(start_year, year+1):
            timesteps += timesteps_in_year(year_tmp, calendar=calendar, dt=dt)
            
        tstart = tend+1  
        tend = tend + timesteps_in_year(year, calendar=calendar, dt=dt)
    # tend = timesteps_in_year(year, calendar=calendar, dt=dt)
    
    return tstart, tend 

def identify_year(tstart, tend, start_year, end_year, calendar='standard', dt=1350):
    if tstart == 0:
        # Simulation hasn't started yet.
        year = start_year
        print('echo First year of simulation is '+str(year))
    else:
        timesteps=0
        for year_tmp in range(start_year, end_year+1):
            timesteps += timesteps_in_year(year_tmp, calendar=calendar, dt=dt)
            
            if timesteps == tend:
                # This year was the last simulation year
                year = year_tmp + 1
                if year <= end_year:
                    print('echo Next year of simulation is '+str(year))
                    break
            if timesteps > tend:
                raise Exception('Something is wrong with the timesteps in your namelist!')

    return year

def update_namelist(tstart, tend, start_year, end_year, repeat, calendar='standard', dt=1350): 
    endstep_filename='/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/scripts/end_timestep.txt'

    # Figure out what run year it is  
    timestep_end = 0
    if repeat!=0:
        with open(endstep_filename,'r') as file:
            timestep_end = int(file.readlines()[0]) 
        if tstart <= timestep_end:
            tstart = 0;
            tend   = 0;
        else: 
            tstart = tstart - timestep_end
            tend   = tend   - timestep_end
        year = identify_year(tstart, tend, start_year, end_year, calendar=calendar, dt=dt)
    else:
        year = identify_year(tstart, tend, start_year, end_year, calendar=calendar, dt=dt)

    # Identify next timestep
    tstart, tend = next_timestep(year, start_year, tstart, tend, calendar=calendar, dt=dt)
    tstart = tstart + timestep_end
    tend   = tend   + timestep_end
    if year == end_year:
        print('echo Finished simulation at year', year, 'and timestep', tend)
        with open(endstep_filename,'w') as file: # write last time step of upcoming run to a file
            file.write(str(tend)) 
    else:
        print(year, tstart, tend)
    
    return tstart, tend 

In [3]:
# Inner function to calculate number of timesteps in a given year
def timesteps_in_year (year, calendar='standard', dt=1350):
    if calendar == '360-day':
        ndays = 360
    elif calendar == 'noleap':
        ndays = 365
    elif calendar == 'standard':
        if year%4 == 0 and (year%100 != 0 or year%400 == 0):
            ndays = 366
        else:
            ndays = 365
    
    return int(ndays*24*3600/dt)